In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customercode/Customer code.xlsx


In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# !pip install fsspec==2024.10.0 --force-reinstall
# !pip install -q torch transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl pacmap datasets langchain-community ragatouille


In [4]:
# !pip install --upgrade langchain langchain-google-genai google-generativeai


In [14]:
import pandas as pd

class ExcelProcessor:
    def __init__(self, file_path):
        """
        Initializes the ExcelProcessor and loads all sheets with dynamic headers.

        Parameters:
        - file_path (str): Path to the Excel file.
        """
        self.file_path = file_path
        self.sheets_data = self._load_excel_with_dynamic_headers()

    def _load_excel_with_dynamic_headers(self):
        """
        Loads an Excel file and automatically detects the header row for each sheet.

        Returns:
        - dict: Dictionary of DataFrames where keys are sheet names.
        """
        xls = pd.ExcelFile(self.file_path)
        sheet_dfs = {}

        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name, header=None, dtype="str")  # Read without header
            unique_counts = df.nunique(axis=1)  # Count unique values per row
            header_row = unique_counts.idxmax()  # Find the row with the most unique values

            # Read again with detected header row
            df = pd.read_excel(xls, sheet_name=sheet_name, header=header_row, dtype="str").reset_index(drop=True)

            # Store DataFrame
            sheet_dfs[sheet_name] = df

        return sheet_dfs
    def summarize_data(self, sheet_name, dimension_cols=None, agg_cols=None, agg_funcs=None, filters=None, agg_dtype=None, contains_filter=False, dropna=True):
        """
        Summarizes data from a given sheet dynamically with optional exact or substring-based filtering.
    
        Parameters:
        - sheet_name (str): Name of the sheet to summarize.
        - dimension_cols (list, optional): Columns used for grouping. If None, aggregates all values.
        - agg_cols (list): Columns to aggregate.
        - agg_funcs (list): Functions to apply (same order as agg_cols).
        - filters (dict, optional): Filtering conditions. Supports exact and contains filtering.
        - agg_dtype (dict, optional): Dictionary to change data types of aggregation columns.
        - contains_filter (bool, optional): If True, applies substring filtering instead of exact matching.
        - dropna (bool, optional): If True (default), removes rows with NaN in dimension_cols before grouping.
    
        Returns:
        - DataFrame: Filtered and summarized DataFrame.
        """
        if sheet_name not in self.sheets_data:
            raise ValueError(f"Sheet '{sheet_name}' not found in the Excel file.")
    
        df = self.sheets_data[sheet_name].copy()  # Copy to avoid modifying the original
    
        # Convert aggregation columns to numeric if specified
        if agg_cols:
            for col in agg_cols:
                df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert non-numeric values to NaN
    
        # Apply filters if provided
        if filters:
            for col, values in filters.items():
                if contains_filter:
                    df = df[df[col].astype(str).apply(lambda x: any(v in x for v in values))]
                else:
                    df = df[df[col].isin(values)]
    
        # Remove NaN values from dimension columns if specified
        if dimension_cols and dropna:
            df = df.dropna(subset=dimension_cols)
    
        # Create aggregation dictionary dynamically
        agg_dict = {col: func for col, func in zip(agg_cols, agg_funcs)}
    
        # Perform groupby aggregation
        if dimension_cols:
            summary = df.groupby(dimension_cols, dropna=not dropna).agg(agg_dict).reset_index()
        else:
            summary = df.agg(agg_dict).to_frame().T  # Aggregate all values (no grouping)
    
        # Change data types for aggregation columns if specified
        if agg_dtype:
            summary = summary.astype(agg_dtype)
    
        return summary
    


In [11]:
task = ExcelProcessor("/kaggle/input/customercode/Customer code.xlsx")
summary_result = task.summarize_data(
    sheet_name="GL",  
    agg_cols = ['Credit'], agg_funcs=  ['sum'], filters = {"VAS_2":["511", "711"]},  # Apply filtering
    contains_filter=False, 
    agg_dtype = {'Credit': 'float'}
)
summary_result

,Credit
0,1.716906e+10


In [15]:
summary_result

,Credit
0,1.716906e+10


In [16]:
task.sheets_data['GL']

,Unnamed: 0,Job ID,Job Type,Type,BS/PL,VAS_1,VAS_2,VAS_3,Specific item,P/L Item,...,LastUpdDt,PymntTerms,PymntTermsDt,LocalAcc,Descr.Account,Long Description,BillTypeId,Folio,CustomerVATId,SupplierVATId
0,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,HO,OH below COGS,6,P/L,635,635,635,Accrued interest,Financial expenses,...,NaN,NaN,NaN,6611600,Lãi vay tài chính,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Site,1,BS,335,335,335,NaN,NaN,...,NaN,NaN,NaN,1688030,Trích trước chi phí lãi vay hi,NaN,NaN,NaN,NaN,NaN
3,NaN,HO,OH below COGS,6,P/L,635,635,635,Accrued interest,Financial expenses,...,NaN,NaN,NaN,6611600,Lãi vay tài chính,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Site,5,BS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5800200,Chuyển khoản nội bộ nhà cung c,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63312,NaN,HO,OH below COGS,4,BS,2421,2421,242,NaN,NaN,...,NaN,NaN,NaN,4860410,Chi phí trả trước Tập đoàn,NaN,NaN,NaN,NaN,NaN
63313,NaN,PKB,Site,7,P/L,5112,5112,511,NaN,Revenues,...,NaN,NaN,NaN,7041200,Doanh thu ước tính theo hình h,NaN,NaN,NaN,NaN,NaN
63314,NaN,a04ccNTT23,Site,7,P/L,5112,5112,511,NaN,Revenues,...,NaN,NaN,NaN,7089050,Phân bổ cho chi nhánh - dịch v,NaN,NaN,NaN,NaN,NaN
63315,NaN,PKB,Site,4,BS,337,337,337,NaN,NaN,...,NaN,NaN,NaN,4180002,Phải thu khách hàng (chưa xuất,NaN,NaN,NaN,NaN,NaN


In [9]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")
secret_value_1 = user_secrets.get_secret("HUGGING_FACE_KEY")
secret_value_2 = user_secrets.get_secret("LLAMA_INDEX_KEY")
secret_value_3 = user_secrets.get_secret("OPEN_AI_KEY")


In [ ]:
genai.configure(api_key=secret_value_0)
model = genai.GenerativeModel('gemini-2.0-flash')

In [25]:
def exec_any_command(command: str):
    """
    Executes a one-line Pandas command on the dataframe:

    Args:
        command: A string containing valid Python code to be executed.

    Returns:
        The output of the executed command.
    """
    local_vars = {'df': task.sheets_data['GL']}
    try:
        if ';' in command:
            # Execute all parts except the last one if it's assignment
            parts = command.split(';')
            for part in parts[:-1]:
                exec(part.strip(), globals(), local_vars)

            last_part = parts[-1].strip()
            if '=' in last_part:
                # Assignment → just exec
                exec(last_part, globals(), local_vars)
                # Try to return the assigned variable
                var_name = last_part.split('=')[0].strip()
                output = local_vars.get(var_name, f"Variable '{var_name}' not found.")
            else:
                # Expression → eval
                output = eval(last_part, globals(), local_vars)

        else:
            output = eval(command, globals(), local_vars)

    except Exception as e:
        output = f"[Error during execution] {str(e)}"

    return output


In [26]:
import json
import random
import string
import re

def extract_tool_call(llm_response) : 
    """From the output text generated by the llm, it'll extract the function(with the arguments) """
    try :
        tool_call = json.loads(llm_response)[0]

    except :
        # Step 1: Extract the JSON-like part using regex
        json_part = re.search(r'\[.*\]', llm_response, re.DOTALL).group(0)

        # Step 2: Convert it to a list of dictionaries
        tool_call = json.loads(json_part)[0]

    return tool_call


def redef_messages(messages, response) :
    """Generates a tool_call_id and append it to the messages prompt-template (as required by Mistral)"""
    tool_call_id = ''.join(random.choices(string.ascii_letters + string.digits, k=9))
    # Append the tool call to the conversation
    messages.append({"role": "assistant", "tool_calls": [{"type": "function", "id": tool_call_id, "function": response}]})
    
    return messages, tool_call_id




def extract_and_exec(messages,tool_call,tool_call_id) : 
    """Executing the defined tool and appending the results of the tool to the message prompt-template """
    function_name = tool_call["name"]   # Extracting the name of the tool (function) from the tool_call dictionary.

    arguments = tool_call["arguments"]  # Extracting the arguments for the function from the tool_call dictionary.

    
    console_output = exec_any_command(**arguments)

    print(console_output)
    messages.append({"role": "tool", "tool_call_id": tool_call_id, "name": f"{function_name}", "content": f"{str(console_output)}"})
    
    return messages, console_output




In [27]:
## For printing in Bold green
TGREEN = '\033[1;32m' 
## For printing in Bold Blue
TBLUE = '\033[1;34m'
## For printing in Black
TBLACK = '\033[30m'

In [28]:
import pandas as pd
import datetime
import os
import subprocess
import google.generativeai as genai

# === Already configured ===
# genai.configure(api_key=secret_value_0)
# model = genai.GenerativeModel('gemini-2.0-flash')

def llm_response_exec(
    user_input,
    df=task.sheets_data['GL'],
    func=exec_any_command,
    export_path=None,
    auto_open=False,
    verbose=True
):
    if verbose:
        print(f">> User's Input : {user_input}\n")

    # 1. Create the system prompt
    system_prompt = f"""
You are a Data Analysis Assistant.
You can execute Python Pandas one-line commands on this dataframe:

{df.head(5).to_markdown()}

For any user query, generate a single-line Pandas expression.
Use exact column names as mentioned. Chain operations if needed.
Only respond in this format:

exec_any_command("PANDAS_CODE")

Additionally, summarize the result in one sentence.
"""

    full_prompt = system_prompt + f"\n\nUser Query: {user_input}"

    # 2. Call Gemini Model
    response = model.generate_content(full_prompt)
    text_response = response.text

    if verbose:
        print(">> Gemini Response:\n")
        print(text_response)

    # 3. Extract command
    import re
    match = re.search(r'exec_any_command\("(.+?)"\)', text_response)
    if not match:
        print("❗️ Could not extract Pandas command.")
        return None

    command = match.group(1)

    # 4. Execute
    result = func(command)

    if verbose:
        print("\n>> Execution Result:\n")
        print(result)

    # 5. Export if DataFrame
    export_done = False
    if isinstance(result, pd.DataFrame):
        if export_path is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            export_path = f"result_{timestamp}.xlsx"

        if export_path.endswith(".csv"):
            result.to_csv(export_path, index=False)
        else:
            result.to_excel(export_path, index=False)

        export_done = True
        if verbose:
            print(f"\n✅ Result exported to: {export_path}")

        if auto_open:
            try:
                if os.name == 'posix':
                    subprocess.call(('open', export_path))
                elif os.name == 'nt':
                    os.startfile(export_path)
            except Exception as e:
                print(f"⚠️ Couldn't open file: {e}")

    # 6. Return
    return {
        "command": command,
        "execution_result": result,
        "llm_response": text_response,
        "exported": export_done,
        "export_path": export_path if export_done else None
    }


In [38]:
result = llm_response_exec(
    "convert VAS_2 to float then filter VAS_2 == '711', change column 'Credit' to float and sum it group by 'Sub Account', the result should not be scientific type ")


>> User's Input : convert VAS_2 to float then filter VAS_2 == '711', change column 'Credit' to float and sum it group by 'Sub Account', the result should not be scientific type 

>> Gemini Response:

```python
import pandas as pd
from io import StringIO

csv_data = """
   Unnamed: 0,Job ID,Job Type,Type,BS/PL,VAS_1,VAS_2,VAS_3,Specific item,P/L Item,Related party,Unnamed: 11,FS-Debit,FS-Credit,Month,Year,For Detai.Job Sheet,Sheet Check Job,Sheet Check analytic,CAT,Entity GL,Account,Account Descr,Analytic,Desc. Analytic,Journal ID,Journal Date,Description,Line Nbr,Journal Line Descr,Ledger,Project,Desc. Project,Journal Line Open Item Key,Status,JournalCode,Journal DocSeqNb,Divisor,Multiplier,Amount,Currency,Base Amount,Base Currency,Sub Account,Sub Account Description,First name,Supervisor,Country,Invoice ID,Invoice Date,Issue Date,Due Date,Entry Code,Desc. Entry,GroupID,Payment Terms,PaymentID,Ref. Payment,PaymentMode,Accounting Date,nv Doc Type,Inv DocSeqNb,Debit,Credit,ValidDate,Jour

<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
